In [ ]:
# Carl Tape, Lion Krischer, Amanda McPherson
# GEOS626, Applied Seismology, University of Alaska Fairbanks

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 5

In [ ]:
# getting the inventory

client = Client("IRIS")

starttime = UTCDateTime(2004, 12, 26, 0, 58, 50)
endtime   = starttime + ( 5 * 24 * 60 * 60 )        # 5 days after the start time

# check the page - https://docs.obspy.org/packages/obspy.core.inventory.html, 
# for detailed information on the inventory object
inv_LHZ = client.get_stations(network="G", station="CAN", channel="LHZ", starttime=starttime, endtime=endtime, level="response")
inv_BHZ = client.get_stations(network="G", station="CAN", channel="BHZ", starttime=starttime, endtime=endtime, level="response")

In [ ]:
# see details about the instrument response

# set inventory for the channel of interest
#inv = inv_LHZ.copy()
inv = inv_BHZ.copy()

print(inv)

In [ ]:
print(inv[0])

In [ ]:
print(inv[0][0])

In [ ]:
print(inv[0][0][0])

In [ ]:
# collect key values from the inventory

samples_per_second = inv[0][0][0].sample_rate
dt = 1/samples_per_second
Nyquist_frequency  = 1 / (2*dt)
print(f'sample rate = {samples_per_second} 1/s, dt = {dt} s, Nyquist frequency = {Nyquist_frequency} Hz \n')

print(inv[0][0][0].response)

In [ ]:
# check the page - https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.plot_response.html,
# for detailed information on plotting response using this method

#inv.plot_response?

In [ ]:
#inv = inv_LHZ.copy()

# plot the instrument response (note: semi-colon is needed to block a repeated plot)
inv.plot_response(min_freq=0.0001);

In [ ]:
# optional: plot the stages separately
#inv[0][0][0].plot(min_freq=0.0001, start_stage=1, end_stage=1, output="VEL");
#inv[0][0][0].plot(min_freq=0.0001, start_stage=2, end_stage=2, output="VEL");
#inv[0][0][0].plot(min_freq=0.0001, start_stage=3, end_stage=3, output="VEL");

In [ ]:
# from here on out, we will specify the specific frequencies for which we will evaluate the instrument response
f = np.logspace(-4.0, 2.0, num=100)

In [ ]:
Id = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='DISP')
Iv = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='VEL')
Ia = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='ACC')

max_Id   = max(Id)
f_Id_max = f[np.nonzero(Id == max_Id)]
max_Iv   = max(Iv)
f_Iv_max = f[np.nonzero(Iv == max_Iv)]
max_Ia   = max(Ia)
f_Ia_max = f[np.nonzero(Ia == max_Ia)]

fig, ax = plt.subplots(3, 2, figsize=(14,16))

ax[0,0].semilogx(f, np.angle(Id)*180/np.pi, color='b')
ax[0,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[0,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[0,1].loglog(f, abs(Id), color='b')
ax[0,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[0,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])
ax[0,1].set_title('(m to counts) max = %.2e at %.2e Hz'% (np.real(max_Id), f_Id_max), fontsize=11)

ax[1,0].semilogx(f, np.angle(Iv)*180/np.pi, color='b')
ax[1,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[1,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[1,1].loglog(f, abs(Iv), color='b')
ax[1,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[1,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])
ax[1,1].set_title('(m/s to counts) max = %.2e at %.2e Hz'% (np.real(max_Iv), f_Iv_max), fontsize=11)

ax[2,0].semilogx(f, np.angle(Ia)*180/np.pi, color='b')
ax[2,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[2,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[2,1].loglog(f, abs(Ia), color='b')
ax[2,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[2,1].set_title('(m/s2 to counts) max = %.2e at %.2e Hz'% (np.real(max_Ia), f_Ia_max), fontsize=11)
ax[2,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

fig.suptitle(f'Network - {inv[0].code}, Station - {inv[0][0].code}, Location - {inv[0][0][0].location_code}, Channel - {inv[0][0][0].code}', fontsize=12)

plt.subplots_adjust(hspace=0.25)

In [ ]:
# superimposing BHZ and LHZ instrument responses for velocity

Iv_BHZ = inv_BHZ[0][0][0].response.get_evalresp_response_for_frequencies(f, output='VEL')
Iv_LHZ = inv_LHZ[0][0][0].response.get_evalresp_response_for_frequencies(f, output='VEL')

fNy_BHZ = (inv_BHZ[0][0][0].sample_rate) / 2
fNy_LHZ = (inv_LHZ[0][0][0].sample_rate) / 2

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

ax1.semilogx(f, np.angle(Iv_BHZ)*180/np.pi, color='b', label='BHZ')
ax1.semilogx(f, np.angle(Iv_LHZ)*180/np.pi, color='r', label='LHZ')
ax1.axvline(fNy_BHZ, c='b', lw=0.5, ls='--', dashes=[12,12])
ax1.axvline(fNy_LHZ, c='r', lw=0.5, ls='--', dashes=[12,12])
ax1.set(xlabel='Frequency (Hz)', ylabel='Phase (degrees)')
ax1.legend(loc='lower left', shadow=True)

ax2.loglog(f, abs(Iv_BHZ)/max(abs(Iv_BHZ)), color='b', label='BHZ')
ax2.loglog(f, abs(Iv_LHZ)/max(abs(Iv_LHZ)), color='r', label='LHZ')
ax2.axvline(fNy_BHZ, c='b', lw=0.5, ls='--', dashes=[12,12])
ax2.axvline(fNy_LHZ, c='r', lw=0.5, ls='--', dashes=[12,12])
ax2.set(xlabel='Frequency (Hz)', ylabel='Normalized Amplitude')
ax2.legend(loc='lower left', shadow=True)

fig.suptitle(f'Network - {inv[0].code}, Station - {inv[0][0].code}, Location - {inv[0][0][0].location_code}', fontsize=12)